In [1]:
qpath = r'C:/Users/rgoix/DEV/quetzal/' # path to quetzal here
data = r'inputs/'
import sys
sys.path.append(qpath + r'quetzal/')

# import class
from quetzal.io.gtfs_reader import importer

### Read GTFS

In [2]:
feed = importer.GtfsImporter(path=data + r'madrid.zip', dist_units='m')
feed.clean()
feed.describe()

,indicator,value
0,agencies,[Empresa Municipal de Transportes]
1,running_services,"[SA, FE, LA, VV, LJ]"
2,timezone,Europe/Madrid
3,start_date,20191210
4,end_date,20201231
5,num_routes,215
6,num_trips,79196
7,num_stops,4724
8,num_shapes,426
9,num_frequencies,78520


Frequency conversion currently work only for one specific service and date, and on one given time period.  
It computes the average headway over this time period.

### Restrict to one date and merge services

In [3]:
feed = feed.restrict(dates=['20191210'])
feed.group_services()

In [4]:
feed.describe()

,indicator,value
0,agencies,[Empresa Municipal de Transportes]
1,running_services,[LA]
2,timezone,Europe/Madrid
3,start_date,20191210
4,end_date,20191210
5,num_routes,210
6,num_trips,35867
7,num_stops,4715
8,num_shapes,420
9,num_frequencies,35647


In [5]:
feed.map_routes(feed.routes.route_id.head())

### Build patterns
In a non-frequency based GTFS, a route has many trips, each corresponding to a start at a specific time. It is necessary to group these trips in order to compute their headway. A group of trips is refered as a pattern.
The default method to build the patterns is by grouping trips based on their ordered list of stop_ids, without knowledge of time or duration.  
Other methods to build the patterns are available: by parent stations, or clusters. See example 4-advanced-patterns.ipynb

In [6]:
feed.build_patterns()

In [7]:
feed.describe()

,indicator,value
0,agencies,[Empresa Municipal de Transportes]
1,running_services,[LA]
2,timezone,Europe/Madrid
3,start_date,20191210
4,end_date,20191210
5,num_routes,210
6,num_trips,35867
7,num_stops,4715
8,num_shapes,420
9,num_frequencies,35647


### Convert to frequencies

In [8]:
time_range = ['06:00:00', '09:00:00']  # time format must be HH:MM:SS
feed_f = feed.convert_to_frequencies(time_range=time_range)

100%|████████████████████████████████████████████████████████████████████████████████| 363/363 [01:26<00:00,  4.20it/s]


In [9]:
feed_f.describe()

,indicator,value
0,agencies,[Empresa Municipal de Transportes]
1,running_services,[LA]
2,timezone,Europe/Madrid
3,start_date,20191210
4,end_date,20191210
5,num_routes,182
6,num_trips,363
7,num_stops,4647
8,num_shapes,363
9,num_frequencies,363


The average headway computed is the interval length divided by the number of trip starts within the interval.
- a trip starting exactly at the start of the time range (6:00:00) is taken into account
- a trip starting exactly at the end of the time range (9:00:00) is not considered.